In [6]:
from openprompt import PromptDataLoader, PromptForClassification
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate, ManualVerbalizer, ManualTemplate
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
import torch

In [2]:
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-large-uncased")

Downloading: 100%|██████████| 571/571 [00:00<00:00, 231kB/s]
Downloading: 100%|██████████| 1.34G/1.34G [00:45<00:00, 29.8MB/s]
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 595kB/s] 
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 19.0kB/s]


In [3]:
def get_dataloader(dataset_split, template, shuffle = False, batch_size=32):
  """ Returns a prompt data load for a given dataset split and template """

  return PromptDataLoader(
    dataset=dataset_split,
    template=template, 
    tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, 
    shuffle=shuffle,
    truncate_method="head",
    decoder_max_length=3,
    batch_size=batch_size,
    teacher_forcing=False,
    predict_eos_token=False,
    max_seq_length=64,
  )

In [4]:
from openprompt.prompts import ManualTemplate
template = ManualTemplate(
    text = '{"placeholder":"text_a"} The pokemon is {"mask"}.',
    tokenizer = tokenizer,
)

In [8]:
# load data
pokemon_descriptions = load_dataset('../data/dataset/', delimiter=';')

NUM_CLASSES = len(np.unique(pokemon_descriptions['train']['labels']))

# train test split
split_pokemon_descriptions = pokemon_descriptions['train'].train_test_split(
    test_size=0.2, shuffle=True)

Using custom data configuration dataset-c155d0d0dfd2c295


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 200.08it/s]


Dataset csv downloaded and prepared to C:/Users/CYBORGX/.cache/huggingface/datasets/csv/dataset-c155d0d0dfd2c295/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 104.95it/s]
Parameter 'generator'=Generator(PCG64) of the transform datasets.arrow_dataset.Dataset.train_test_split couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
